In [54]:
# %pip install --upgrade protobuf
# %pip install --upgrade tensorflow      ## running this at the top after restarting the kernel deals with the import tensorflow error that occured in the later part of the code .

In [55]:
import pandas as pd
import numpy as np

In [56]:
df = pd.read_csv('diabetes.csv')

In [57]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [58]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [59]:
x = df.iloc[:,:-1].values
y =df.iloc[:,-1].values

In [60]:
x

array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]],
      shape=(768, 8))

In [61]:
x.shape

(768, 8)

In [62]:
y.shape

(768,)

In [63]:
from sklearn.preprocessing import StandardScaler

In [64]:
scaler = StandardScaler()

In [65]:
x = scaler.fit_transform(x)

In [66]:
x.shape

(768, 8)

In [67]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2 , random_state=42)

In [68]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [69]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim = 8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

c:\Users\KIIT0001\Desktop\Deep_learning\venv\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [70]:
model.fit(x_train , y_train ,batch_size=32 , epochs=10,validation_data=(x_test,y_test))

Epoch 1/10


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6889 - loss: 0.6047 - val_accuracy: 0.7792 - val_loss: 0.5558
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7345 - loss: 0.5620 - val_accuracy: 0.8312 - val_loss: 0.5245
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7606 - loss: 0.5319 - val_accuracy: 0.8182 - val_loss: 0.5057
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7655 - loss: 0.5125 - val_accuracy: 0.8247 - val_loss: 0.4929
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7736 - loss: 0.4983 - val_accuracy: 0.8117 - val_loss: 0.4858
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7769 - loss: 0.4868 - val_accuracy: 0.8117 - val_loss: 0.4809
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7736 - loss: 0.4785 - val_accuracy: 0.8052 - val_loss: 0.4771
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7785 - loss: 0.4717 - val_accuracy: 0.8052 - val_loss: 0.4738
Epo

The main problem here is till now we were initializing the neurons,optimizers,activation based on our own intuition .....but we can automate it using a library known as keras tuner 

# 1. how to select appropriate optimizer
# 2. no of nodes in a layer
# 3. select number of layers
# 4. all in one model

In [71]:
import kerastuner as kt

# How to select appropriate optimizer

In [72]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(32,activation='relu',input_dim = 8))
    model.add(Dense(1,activation='sigmoid'))
    optimizer = hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta'])

    model.compile(optimizer = optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return model


In [73]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5)

Reloading Tuner from .\untitled_project\tuner0.json


In [74]:
tuner.search(x_train,y_train,epochs = 5 , validation_data = (x_test,y_test))

In [75]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [76]:
model = tuner.get_best_models(num_models=1)[0]

c:\Users\KIIT0001\Desktop\Deep_learning\venv\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [77]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [78]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7443 - loss: 0.5125 - val_accuracy: 0.7662 - val_loss: 0.5502
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7622 - loss: 0.4949 - val_accuracy: 0.7662 - val_loss: 0.5411
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7655 - loss: 0.4842 - val_accuracy: 0.7662 - val_loss: 0.5327
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7736 - loss: 0.4762 - val_accuracy: 0.7727 - val_loss: 0.5271
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7801 - loss: 0.4690 - val_accuracy: 0.7727 - val_loss: 0.5213
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7801 - loss: 0.4636 - val_accuracy: 0.7727 - val_loss: 0.5180
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7801 - loss: 0.4593 - val_accuracy: 0.7597 - val_loss: 0.5160
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7818 - loss: 0.4555 - val_accuracy: 0.759

# how to get number of nodes in a layer

In [79]:
def build_model(hp):
    model = Sequential()
    units = hp.Int('units',min_value = 8,max_value = 128,step=8)
    model.add(Dense(units=units,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

    return model

In [80]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='mydir')

Reloading Tuner from mydir\untitled_project\tuner0.json


In [81]:
tuner.search(x_train, y_train,epochs = 5,validation_data= (x_test,y_test))

In [82]:
tuner.get_best_hyperparameters()[0].values

{'units': 72}

In [83]:
model = tuner.get_best_models(num_models=1)[0]

In [84]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7687 - loss: 0.4663  
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7785 - loss: 0.4566 
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7704 - loss: 0.4512 
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7736 - loss: 0.4481 
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7769 - loss: 0.4444 
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7866 - loss: 0.4417 
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7736 - loss: 0.4394 
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7785 - loss: 0.4377 
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7769 - loss: 0.4355 
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7785 - loss: 0.4344 
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7785 - loss: 0.4330 
Epoch 18/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3m

# select number of layers

In [85]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(72,activation='relu',input_dim = 8))
    for i in range (hp.Int('num_layers',min_value = 1 , max_value = 10)):
        model.add(Dense(72,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

    return model


In [86]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory = 'mydir',
                        project_name = 'num_layers')

Reloading Tuner from mydir\num_layers\tuner0.json


In [87]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [88]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6}

In [89]:
model = tuner.get_best_models(num_models=1)[0]

c:\Users\KIIT0001\Desktop\Deep_learning\venv\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [90]:
model.fit(x_train,y_train,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.7801 - loss: 0.4604 - val_accuracy: 0.7403 - val_loss: 0.5268
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7964 - loss: 0.4307 - val_accuracy: 0.7727 - val_loss: 0.5516
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8013 - loss: 0.4112 - val_accuracy: 0.7727 - val_loss: 0.5485
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8078 - loss: 0.4147 - val_accuracy: 0.7468 - val_loss: 0.5699
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8257 - loss: 0.3986 - val_accuracy: 0.7403 - val_loss: 0.6000
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8225 - loss: 0.3922 - val_accuracy: 0.7403 - val_loss: 0.5433
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8241 - loss: 0.3883 - val_accuracy: 0.7143 - val_loss: 0.5866
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8355 - loss: 0.3568 - val_accuracy: 0.65


# All in one

In [93]:
def build_model(hp):
    model = Sequential()
    counter = 0

    for i in range(hp.Int('num_layers',min_value =1,max_value=10)):
        if counter == 0:
            model.add(
                Dense(
                    hp.Int('units' + str(i) , min_value = 8 , max_value = 128 , step = 8),
                   
                    input_dim = 8
                )
            )

        else :

            model.add(
                Dense(
                    hp.Int('units' + str(i) , min_value = 8 , max_value = 128 , step = 8),
                    optimizer = hp.Choice('activation' + str(i),values=['relu','tanh','sigmoid'])
                    
                )
                
            )
            counter += 1
    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadelta']),
                  loss = 'binary_crossentropy',
                  metrics=['accuracy'])  



    return model   


In [94]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory="mydir",
                        project_name = 'final')

In [95]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.7532467246055603

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 09s


In [97]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 9,
 'units0': 72,
 'optimizer': 'rmsprop',
 'units1': 8,
 'units2': 8,
 'units3': 8,
 'units4': 8,
 'units5': 8,
 'units6': 8,
 'units7': 8,
 'units8': 8}

In [98]:
model = tuner.get_best_models(num_models=1)[0]

c:\Users\KIIT0001\Desktop\Deep_learning\venv\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\KIIT0001\Desktop\Deep_learning\venv\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [100]:
model.fit(x_train,y_train,epochs=200,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7606 - loss: 0.4829 - val_accuracy: 0.7403 - val_loss: 0.5181
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7720 - loss: 0.4798 - val_accuracy: 0.7597 - val_loss: 0.5098
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7622 - loss: 0.4762 - val_accuracy: 0.7468 - val_loss: 0.5188
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7769 - loss: 0.4777 - val_accuracy: 0.7792 - val_loss: 0.5136
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7687 - loss: 0.4754 - val_accuracy: 0.7662 - val_loss: 0.5114
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7687 - loss: 0.4750 - val_accuracy: 0.7338 - val_loss: 0.5270
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7736 - loss: 0.4767 - val_accuracy: 0.7338 - val_loss: 0.5167
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7736 - loss: 0.4772 - val_accuracy: 0.7